In [10]:
from datetime import datetime, timedelta
import pandas_datareader as pdr
from finta import TA
import pandas as pd


class EquityData():
    
    """       
    A class used to get a data frame with trading data and finacial indicators 

    Attributes
    ----------
    ticker_list : Optional[list[str]]
        A list of tickers to retrieve end of day trading data
    indicator_list : Optional[list[str]]
        A list of indicators to calculate for each security in the ticker_list
        
    Examples
    --------
    e = EquityData()
    print(e.df)
    
    ticker_list = ['UBER']
    indicator_list = ['EMA']
    e = EquityData(ticker_list, indicator_list)
    print(e.df)
    
    """
    
    def __init__(self, ticker_list= None, indicator_list= None):

        self.start_date = datetime.now().date() - timedelta(days=20)
        self.end_date = datetime.now().date()
        if ticker_list == None:
            self.ticker_list = ['AAPL', 'TSLA']
        else:
            self.ticker_list = ticker_list
        if indicator_list == None:
            self.indicator_list = ['SMA', 'EMA']
        else:
            self.indicator_list = indicator_list
        self.df = pd.DataFrame([])
        self.get_eod_data(self)

    @staticmethod
    def get_eod_data(self):

        """
        Get EOD data for a list of tickers
        """
        eod_df = pd.DataFrame([])

        for t in self.ticker_list:
            results = pdr.DataReader(t, 'yahoo', self.start_date, self.end_date)
            results['ticker'] = t
            eod_df = eod_df.append(results)

        # Calc the list of indicators for each set of EOD data
        eod_df = eod_df.groupby(['ticker']).apply(lambda x: self.compute_analytics(x, self.indicator_list)).reset_index()

        eod_df = self.clean_data(eod_df)
        self.df = eod_df

    @staticmethod
    def compute_analytics(df, indicators):

        """
        Get calculations for each ticker in the df
        """

        for indicator in indicators:
            # Using eval so we can iterate over a list of indicators
            data = eval('TA.' + indicator + '(df)')

            # TA returns a set -> so we convert that to a data frame
            data = data.to_frame()

            # The columns names are too long so I split the string
            # and join that with the indicator name for my indicator column name
            first_string = data.columns[0].split()[0]
            data = data.rename(columns={data.columns[0]: first_string})
            data = data.add_prefix(indicator + '_')

            # Merge data frames based on the date
            df = df.merge(data, left_index=True, right_index=True)

        return df

    @staticmethod
    def clean_data(df):
        
        """
        Update column names to follow PEP8 and sort values
        """
        
        df = df.rename(columns={'Date': 'date', 'High': 'high', 'Low': 'low', 'Open': 'open',
                                'Close': 'close', 'Volume': 'vol', 'Adj Close': 'adj_close',
                                'SMA_41': 'sma_41', 'EMA_9': 'ema_9'})
        df = df.sort_values(by=['ticker', 'date'])
        return df


In [ ]:
# Optional Params
# ticker_list = ['UBER']
# indicator_list = ['EMA']
# obj = EquityData(ticker_list, indicator_list)
# print(obj.df)

obj = EquityData()
print(obj.df)